# Part 1 - Scraping Data from Wikipedia (Pandas Method)

### Imports

In [1]:
#!pip3 install pandas lxml
import pandas as pd

### Loading html Table into Dataframe

In [2]:
df=pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")[0]
df

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
...,...,...,...
175,M5Z,Not assigned,Not assigned
176,M6Z,Not assigned,Not assigned
177,M7Z,Not assigned,Not assigned
178,M8Z,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,..."


### Cleaning the Data

In [3]:
df.describe()

,Postal Code,Borough,Neighbourhood
count,180,180,180
unique,180,11,100
top,M9L,Not assigned,Not assigned
freq,1,77,77


In [4]:
# Removing Postal Codes that are not assigned a borough
postal_code = df[df.Borough != 'Not assigned'].reset_index(drop = True)
postal_code.head(12)

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [5]:
postal_code.shape

(103, 3)

# Part 2 - Adding Latitude and Longitiude to Dataframe

### Imports 

In [6]:
#!pip3 install geocoder geopy
import geocoder
from geopy.geocoders import Nominatim

### Geocoder only produced "None", daily limit reached! 

In [7]:
#lat_lng_coords = None

# loop until you get the coordinates
#while(lat_lng_coords is None):
  #g = geocoder.google('{}, Toronto, Ontario'.format(df1['Postal Code']))
  #lat_lng_coords = g.latlng

#latitude = lat_lng_coords[0]
#longitude = lat_lng_coords[1]

### Reading CSV into Single Dataframe

In [8]:
lat_lng_coords = pd.read_csv('Geospatial_Coordinates.csv')
print(lat_lng_coords)

    Postal Code   Latitude  Longitude
0           M1B  43.806686 -79.194353
1           M1C  43.784535 -79.160497
2           M1E  43.763573 -79.188711
3           M1G  43.770992 -79.216917
4           M1H  43.773136 -79.239476
..          ...        ...        ...
98          M9N  43.706876 -79.518188
99          M9P  43.696319 -79.532242
100         M9R  43.688905 -79.554724
101         M9V  43.739416 -79.588437
102         M9W  43.706748 -79.594054

[103 rows x 3 columns]


In [9]:
coordinates = pd.DataFrame(lat_lng_coords)
coordinates

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [10]:
# Merging new dataframe with existing dataframe on "Postal Code"
Toronto_data = postal_code.merge(coordinates, on = 'Postal Code')
Toronto_data.head(12)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [11]:
Toronto_data.shape

(103, 5)

# Part 3 - Clustering and Visualizing Toronto Neighborhoods with Folium

### Imports

In [12]:
#!pip3 install requests numpy folium sklearn matplotlib
import requests
import folium
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans

### Foursquare API

In [13]:
# @hidden_cell
CLIENT_ID = '0CPU45EULL2XDSWMCF5MDTRKXVMSHXVIFXQBCNKBHXFZQCSD' # your Foursquare ID
CLIENT_SECRET = 'C353QCCWWAWVRBH5DL2QIBNI30QPKKQBF3Y5OSKMKZZE5LBU' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

In [14]:
# Function to obtain local venues and print neighborhood names
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat,
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [15]:
Toronto_venues = getNearbyVenues(names=Toronto_data['Neighbourhood'],
                                  latitudes=Toronto_data['Latitude'],
                                  longitudes=Toronto_data['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

### Examining Function Results

In [16]:
print(Toronto_venues.shape)
Toronto_venues.head()

(2117, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


In [17]:
Toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Alderwood, Long Branch",8,8,8,8,8,8
"Bathurst Manor, Wilson Heights, Downsview North",23,23,23,23,23,23
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",25,25,25,25,25,25
...,...,...,...,...,...,...
"Willowdale, Willowdale East",34,34,34,34,34,34
"Willowdale, Willowdale West",4,4,4,4,4,4
Woburn,3,3,3,3,3,3


In [18]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 268 uniques categories.


### Calculating Most Common Venues in each Neighborhood

In [19]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighbourhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

,Neighbourhood,Accessories Store,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Truck Stop,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
Toronto_onehot.shape

(2117, 269)

In [21]:
# Creating dataframe with grouped neighborhoods
Toronto_grouped = Toronto_onehot.groupby('Neighbourhood').mean().reset_index()
Toronto_grouped

,Neighbourhood,Accessories Store,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Truck Stop,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.04,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,"Willowdale, Willowdale East",0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,...,0.0,0.0,0.0,0.029412,0.0,0.0,0.0,0.0,0.0,0.0
91,"Willowdale, Willowdale West",0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
92,Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
93,Woodbine Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
Toronto_grouped.shape

(95, 269)

In [23]:
# For Loop to list Top 5 Venues for each neighborhood
num_top_venues = 5

for hood in Toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = Toronto_grouped[Toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0               Skating Rink  0.25
1             Breakfast Spot  0.25
2                     Lounge  0.25
3  Latin American Restaurant  0.25
4          Accessories Store  0.00


----Alderwood, Long Branch----
          venue  freq
0   Pizza Place  0.25
1          Pool  0.12
2           Pub  0.12
3   Coffee Shop  0.12
4  Skating Rink  0.12


----Bathurst Manor, Wilson Heights, Downsview North----
            venue  freq
0            Bank  0.09
1     Coffee Shop  0.09
2        Pharmacy  0.04
3   Shopping Mall  0.04
4  Sandwich Place  0.04


----Bayview Village----
                 venue  freq
0  Japanese Restaurant  0.25
1                 Bank  0.25
2   Chinese Restaurant  0.25
3                 Café  0.25
4          Music Venue  0.00


----Bedford Park, Lawrence Manor East----
                     venue  freq
0           Sandwich Place  0.08
1       Italian Restaurant  0.08
2              Coffee Shop  0.08
3  Comfort Food Restaurant  0

               venue  freq
0         Hobby Shop   0.2
1  Convenience Store   0.2
2   Department Store   0.2
3        Coffee Shop   0.2
4     Discount Store   0.2


----Kensington Market, Chinatown, Grange Park----
                           venue  freq
0                           Café  0.08
1                    Coffee Shop  0.06
2             Mexican Restaurant  0.05
3          Vietnamese Restaurant  0.05
4  Vegetarian / Vegan Restaurant  0.05


----Kingsview Village, St. Phillips, Martin Grove Gardens, Richview Gardens----
                      venue  freq
0            Sandwich Place   1.0
1         Accessories Store   0.0
2            Massage Studio   0.0
3            Medical Center   0.0
4  Mediterranean Restaurant   0.0


----Lawrence Manor, Lawrence Heights----
                    venue  freq
0          Clothing Store  0.50
1       Accessories Store  0.08
2             Event Space  0.08
3                Boutique  0.08
4  Furniture / Home Store  0.08


----Lawrence Park----
       

                   venue  freq
0           Hockey Arena  0.25
1  Portuguese Restaurant  0.25
2            Coffee Shop  0.25
3      French Restaurant  0.25
4      Accessories Store  0.00


----West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale----
                      venue  freq
0       Filipino Restaurant   1.0
1             Movie Theater   0.0
2            Massage Studio   0.0
3            Medical Center   0.0
4  Mediterranean Restaurant   0.0


----Westmount----
                venue  freq
0         Pizza Place  0.17
1         Coffee Shop  0.17
2      Discount Store  0.17
3  Chinese Restaurant  0.17
4        Intersection  0.17


----Weston----
                       venue  freq
0              Jewelry Store   0.5
1                       Park   0.5
2          Accessories Store   0.0
3  Middle Eastern Restaurant   0.0
4        Monument / Landmark   0.0


----Wexford, Maryvale----
                       venue  freq
0                     Bakery   0.2
1               

### Adding Top 5 Venues to Dataframe

In [24]:
# Function to add top venues
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [25]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = Toronto_grouped['Neighbourhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Agincourt,Skating Rink,Breakfast Spot,Lounge,Latin American Restaurant,Accessories Store
1,"Alderwood, Long Branch",Pizza Place,Pool,Pub,Coffee Shop,Skating Rink
2,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Coffee Shop,Pharmacy,Shopping Mall,Sandwich Place
3,Bayview Village,Japanese Restaurant,Bank,Chinese Restaurant,Café,Music Venue
4,"Bedford Park, Lawrence Manor East",Sandwich Place,Italian Restaurant,Coffee Shop,Comfort Food Restaurant,Pharmacy


In [26]:
neighborhoods_venues_sorted.shape

(95, 6)

### Creating Clusters and Adding to Toronto Map

In [27]:
# set number of clusters
kclusters = 12

Toronto_grouped_clustering = Toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_ 

array([ 5,  1,  5,  5,  5,  5,  5,  5,  5,  5,  2,  3,  5,  3,  5,  5,  1,
        5,  5,  5,  5,  5,  5,  5,  5,  5,  2,  5,  5,  5,  5,  5,  1,  5,
        5,  5,  5,  5,  1,  8,  5,  5,  3,  5, 10,  3,  5,  5,  5, 11,  2,
        5,  6,  5,  5,  5,  5,  3,  8,  5,  1,  0,  3,  5,  5,  2,  5,  4,
        5,  9,  1,  1,  5,  5,  3,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,
        3,  7,  3,  2,  5,  5,  1,  3,  5,  2])

In [28]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_merged = Toronto_data

# merge Toronto_grouped with Toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

Toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,0.0,Park,Food & Drink Shop,Accessories Store,Middle Eastern Restaurant,Monument / Landmark
1,M4A,North York,Victoria Village,43.725882,-79.315572,3.0,Hockey Arena,Portuguese Restaurant,Coffee Shop,French Restaurant,Accessories Store
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,5.0,Coffee Shop,Park,Café,Bakery,Pub
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,3.0,Clothing Store,Accessories Store,Event Space,Boutique,Furniture / Home Store
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,3.0,Coffee Shop,Sushi Restaurant,Yoga Studio,Theater,Smoothie Shop


In [29]:
# Removing all instances of missing data and converting "Cluster Labels" to int
Toronto_merged = Toronto_merged.dropna(axis=0).reset_index(drop = True)
Toronto_merged['Cluster Labels'] = Toronto_merged['Cluster Labels'].astype(int)
Toronto_merged

,Postal Code,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,0,Park,Food & Drink Shop,Accessories Store,Middle Eastern Restaurant,Monument / Landmark
1,M4A,North York,Victoria Village,43.725882,-79.315572,3,Hockey Arena,Portuguese Restaurant,Coffee Shop,French Restaurant,Accessories Store
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636,5,Coffee Shop,Park,Café,Bakery,Pub
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763,3,Clothing Store,Accessories Store,Event Space,Boutique,Furniture / Home Store
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494,3,Coffee Shop,Sushi Restaurant,Yoga Studio,Theater,Smoothie Shop
...,...,...,...,...,...,...,...,...,...,...,...
94,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944,5,Pool,River,Smoke Shop,Accessories Store,Middle Eastern Restaurant
95,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160,5,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Gay Bar,Restaurant
96,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558,5,Light Rail Station,Pizza Place,Auto Workshop,Spa,Fast Food Restaurant
97,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509,8,Baseball Field,Accessories Store,Miscellaneous Shop,Moroccan Restaurant,Monument / Landmark


In [30]:
# Folium map information
geolocator = Nominatim(user_agent="Toronto_explorer")
location = geolocator.geocode('Toronto, CA')
latitude = location.latitude
longitude = location.longitude

In [31]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighbourhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster - 1],
        fill=True,
        fill_color=rainbow[cluster - 1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Identification of each Cluster Grouping 

### Cluster 1: Food & Drink Shop

In [32]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 0, Toronto_merged.columns[[2] + list(range(6, Toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Parkwoods,Park,Food & Drink Shop,Accessories Store,Middle Eastern Restaurant,Monument / Landmark


### Cluster 2: Pizza Place

In [33]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 1, Toronto_merged.columns[[2] + list(range(6, Toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
7,"Parkview Hill, Woodbine Gardens",Pizza Place,Athletics & Sports,Pharmacy,Pet Store,Breakfast Spot
9,Glencairn,Pizza Place,Pub,Japanese Restaurant,Bakery,Men's Store
31,Scarborough Village,Playground,Pizza Place,Pharmacy,Pet Store,Martial Arts School
48,Humber Summit,Pizza Place,Intersection,Furniture / Home Store,Mobile Phone Shop,Moroccan Restaurant
69,"Willowdale, Willowdale West",Pizza Place,Pharmacy,Butcher,Coffee Shop,Miscellaneous Shop
79,"Clarks Corners, Tam O'Shanter, Sullivan",Pharmacy,Pizza Place,Noodle House,Fried Chicken Joint,Bank
86,"South Steeles, Silverstone, Humbergate, Jamest...",Grocery Store,Pharmacy,Pizza Place,Beer Store,Fried Chicken Joint
90,"Alderwood, Long Branch",Pizza Place,Pool,Pub,Coffee Shop,Skating Rink


### Cluster 3: Park

In [34]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 2, Toronto_merged.columns[[2] + list(range(6, Toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
20,Caledonia-Fairbanks,Park,Women's Store,Accessories Store,Mexican Restaurant,Monument / Landmark
34,"East Toronto, Broadview North (Old East York)",Park,Convenience Store,Accessories Store,Miscellaneous Shop,Moroccan Restaurant
61,Weston,Jewelry Store,Park,Accessories Store,Middle Eastern Restaurant,Monument / Landmark
63,York Mills West,Park,Convenience Store,Accessories Store,Miscellaneous Shop,Moroccan Restaurant
82,"Milliken, Agincourt North, Steeles East, L'Amo...",Playground,Intersection,Park,Miscellaneous Shop,Moroccan Restaurant
88,Rosedale,Park,Playground,Trail,Accessories Store,Middle Eastern Restaurant


### Cluster 4: Coffee Shop

In [35]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 3, Toronto_merged.columns[[2] + list(range(6, Toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
1,Victoria Village,Hockey Arena,Portuguese Restaurant,Coffee Shop,French Restaurant,Accessories Store
3,"Lawrence Manor, Lawrence Heights",Clothing Store,Accessories Store,Event Space,Boutique,Furniture / Home Store
4,"Queen's Park, Ontario Provincial Government",Coffee Shop,Sushi Restaurant,Yoga Studio,Theater,Smoothie Shop
21,Woburn,Coffee Shop,Korean BBQ Restaurant,Accessories Store,Miscellaneous Shop,Moroccan Restaurant
23,Central Bay Street,Coffee Shop,Sandwich Place,Italian Restaurant,Café,Thai Restaurant
33,"Northwood Park, York University",Massage Studio,Caribbean Restaurant,Bar,Furniture / Home Store,Coffee Shop
37,"Kennedy Park, Ionview, East Birchmount Park",Hobby Shop,Convenience Store,Department Store,Coffee Shop,Discount Store
67,Westmount,Pizza Place,Coffee Shop,Discount Store,Chinese Restaurant,Intersection
73,Canada Post Gateway Processing Centre,Coffee Shop,Hotel,Mediterranean Restaurant,Gym,Gas Station
87,"Steeles West, L'Amoreaux West",Coffee Shop,Fast Food Restaurant,Pizza Place,Breakfast Spot,Burger Joint


### Cluster 5: Bar

In [36]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 4, Toronto_merged.columns[[2] + list(range(6, Toronto_merged.shape[1]))]].head(20)

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
11,"Rouge Hill, Port Union, Highland Creek",Bar,Construction & Landscaping,Home Service,Accessories Store,Miscellaneous Shop


### Cluster 6: Gym/Athletics

In [44]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 5, Toronto_merged.columns[[2] + list(range(6, Toronto_merged.shape[1]))]].head(20)

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
2,"Regent Park, Harbourfront",Coffee Shop,Park,Café,Bakery,Pub
6,Don Mills,Gym,Restaurant,Beer Store,Coffee Shop,Japanese Restaurant
8,"Garden District, Ryerson",Clothing Store,Coffee Shop,Cosmetics Shop,Middle Eastern Restaurant,Hotel
12,Don Mills,Gym,Restaurant,Beer Store,Coffee Shop,Japanese Restaurant
13,Woodbine Heights,Skating Rink,Curling Ice,Spa,Park,Beer Store
14,St. James Town,Coffee Shop,Café,Cocktail Bar,Gastropub,American Restaurant
15,Humewood-Cedarvale,Tennis Court,Field,Hockey Arena,Trail,Miscellaneous Shop
16,"Eringate, Bloordale Gardens, Old Burnhamthorpe...",Park,Pizza Place,Pharmacy,Pet Store,Shopping Plaza
17,"Guildwood, Morningside, West Hill",Breakfast Spot,Medical Center,Mexican Restaurant,Intersection,Electronics Store
18,The Beaches,Pub,Health Food Store,Neighborhood,Trail,Middle Eastern Restaurant


### Cluster 7: Summer Camp

In [38]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 6, Toronto_merged.columns[[2] + list(range(6, Toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
80,"Moore Park, Summerhill East",Summer Camp,Accessories Store,Mexican Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant


### Cluster 8: Filipino Restaurant	

In [39]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 7, Toronto_merged.columns[[2] + list(range(6, Toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
10,"West Deane Park, Princess Gardens, Martin Grov...",Filipino Restaurant,Movie Theater,Massage Studio,Medical Center,Mediterranean Restaurant


### Cluster 9: Baseball Field

In [40]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 8, Toronto_merged.columns[[2] + list(range(6, Toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
54,"Humberlea, Emery",Construction & Landscaping,Baseball Field,Accessories Store,Miscellaneous Shop,Moroccan Restaurant
97,"Old Mill South, King's Mill Park, Sunnylea, Hu...",Baseball Field,Accessories Store,Miscellaneous Shop,Moroccan Restaurant,Monument / Landmark


### Cluster 10: Convenience Store

In [41]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 9, Toronto_merged.columns[[2] + list(range(6, Toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
60,"Runnymede, The Junction North",Convenience Store,Breakfast Spot,Bus Line,Accessories Store,Mobile Phone Shop


### Cluster 11: Sandwich Place

In [42]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 10, Toronto_merged.columns[[2] + list(range(6, Toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
74,"Kingsview Village, St. Phillips, Martin Grove ...",Sandwich Place,Accessories Store,Massage Studio,Medical Center,Mediterranean Restaurant


### Cluster 12: Fast Food Restaurant	

In [43]:
Toronto_merged.loc[Toronto_merged['Cluster Labels'] == 11, Toronto_merged.columns[[2] + list(range(6, Toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
5,"Malvern, Rouge",Fast Food Restaurant,Movie Theater,Massage Studio,Medical Center,Mediterranean Restaurant
